Drivers for the paper requirements.

In [1]:
import Lib_TrafficFlow as Lib_TrafficFlow

In [2]:
# import for visualization
import matplotlib
get_ipython().run_line_magic('matplotlib', 'inline')    # for inline plot

In [3]:
# import timer for benchmarking
import time
START_TIME = time.time()
def get_time():
    global START_TIME
    return_time = time.time() - START_TIME
    START_TIME = time.time()
    return return_time

Load the datasets.
For this, we would be having a few datasets:
- traffic flow datasets
- point of interest datasets, let us use petrol stations for now

These datasets would be loaded using dictionary for easy access

In [4]:
# get the filenames
dict_filenames_trafficflow = Lib_TrafficFlow.dict_trafficflow_paper
dict_filenames_poi = Lib_TrafficFlow.dict_poi
# load the data into dataframes
dict_df_trafficflow = {}
dict_df_poi = {}
for current_key in dict_filenames_trafficflow.keys():
    current_filename = dict_filenames_trafficflow[current_key]
    dict_df_trafficflow[current_key] = Lib_TrafficFlow.load_into_dataframe(current_filename)
for current_key in dict_filenames_poi.keys():
    current_filename = dict_filenames_poi[current_key]
    dict_df_poi[current_key] = Lib_TrafficFlow.load_into_dataframe(current_filename)

=======================================================================================
# Traffic Flow
=======================================================================================

In [5]:
# preprocess the flow
# also adjust the start/end points
for current_key in dict_df_trafficflow.keys():
    current_df = dict_df_trafficflow[current_key]
    current_df = Lib_TrafficFlow.preprocess_dataframe_trafficflow(current_df)
    dict_df_trafficflow[current_key] = current_df

In [6]:
# print out the details
for current_key in dict_df_trafficflow.keys():
    print(current_key)
    current_df = dict_df_trafficflow[current_key]
    Lib_TrafficFlow.print_details(current_df)
    # input("continue?")

KL_Selangor_thursday_0800
<class 'pandas.core.frame.DataFrame'>
Int64Index: 120780 entries, 0 to 122687
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   geometry     120780 non-null  category      
 1   TMC_PC       120780 non-null  category      
 2   TMC_DE       120780 non-null  category      
 3   TMC_QD       120780 non-null  category      
 4   TMC_LE       120780 non-null  float64       
 5   FC           120780 non-null  category      
 6   CF_SP        120780 non-null  float64       
 7   CF_SU        120780 non-null  float64       
 8   CF_FF        120780 non-null  float64       
 9   CF_JF        120780 non-null  float64       
 10  CF_CN        120780 non-null  float64       
 11  PBT          120780 non-null  datetime64[ns]
 12  DE           120780 non-null  category      
 13  endpoints    120780 non-null  category      
 14  start_point  120780 non-null  category      
 15  end_poin

In [7]:
# have the uniques
for current_key in dict_df_trafficflow.keys():
    current_df = dict_df_trafficflow[current_key]
    current_df = Lib_TrafficFlow.remove_duplicate_by_timestamp(current_df)
    dict_df_trafficflow[current_key] = current_df

In [8]:
# print out the details
for current_key in dict_df_trafficflow.keys():
    print(current_key)
    current_df = dict_df_trafficflow[current_key]
    Lib_TrafficFlow.print_details(current_df)
    # input("continue?")

KL_Selangor_thursday_0800
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9920 entries, 0 to 110504
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   geometry     9920 non-null   category      
 1   TMC_PC       9920 non-null   category      
 2   TMC_DE       9920 non-null   category      
 3   TMC_QD       9920 non-null   category      
 4   TMC_LE       9920 non-null   float64       
 5   FC           9920 non-null   category      
 6   CF_SP        9920 non-null   float64       
 7   CF_SU        9920 non-null   float64       
 8   CF_FF        9920 non-null   float64       
 9   CF_JF        9920 non-null   float64       
 10  CF_CN        9920 non-null   float64       
 11  PBT          9920 non-null   datetime64[ns]
 12  DE           9920 non-null   category      
 13  endpoints    9920 non-null   category      
 14  start_point  9920 non-null   category      
 15  end_point    9920 non-null 

We still have some inconsistencies, so we now will proceed to clean it further


In [9]:
dict_df_trafficflow = Lib_TrafficFlow.remove_duplicate_from_dataframes(argv_dfs=dict_df_trafficflow, argv_colnames=["start_point", "end_point"])

In [10]:
# print out the details
for current_key in dict_df_trafficflow.keys():
    print(current_key)
    current_df = dict_df_trafficflow[current_key]
    Lib_TrafficFlow.print_details(current_df)

KL_Selangor_thursday_0800
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9915 entries, 0 to 110504
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   geometry     9915 non-null   category      
 1   TMC_PC       9915 non-null   category      
 2   TMC_DE       9915 non-null   category      
 3   TMC_QD       9915 non-null   category      
 4   TMC_LE       9915 non-null   float64       
 5   FC           9915 non-null   category      
 6   CF_SP        9915 non-null   float64       
 7   CF_SU        9915 non-null   float64       
 8   CF_FF        9915 non-null   float64       
 9   CF_JF        9915 non-null   float64       
 10  CF_CN        9915 non-null   float64       
 11  PBT          9915 non-null   datetime64[ns]
 12  DE           9915 non-null   category      
 13  endpoints    9915 non-null   category      
 14  start_point  9915 non-null   category      
 15  end_point    9915 non-null 

After removing the duplicates, we are left with:
- 9915 routes in the dataset
- a start point/ end point would occur a maximum of 4 times, telling us a max degree of 4
- there are 8029 unique start points
- there are 7744 unique end points

In [11]:
# get the frequency for some start/end point
current_key = next(iter(dict_df_trafficflow.keys()))
print(current_key)
current_df = dict_df_trafficflow[current_key]
current_df_freq_startpoint = current_df["start_point"].value_counts().reset_index()
current_df_freq_endpoint = current_df["end_point"].value_counts().reset_index()
current_df_freq_startpoint.columns = ["point", "frequency"]
current_df_freq_endpoint.columns = ["point", "frequency"]
Lib_TrafficFlow.print_details(current_df_freq_startpoint)
Lib_TrafficFlow.print_details(current_df_freq_endpoint)

KL_Selangor_thursday_0800
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8032 entries, 0 to 8031
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   point      8032 non-null   category
 1   frequency  8032 non-null   int64   
dtypes: category(1), int64(1)
memory usage: 461.3 KB
None
                    point    frequency
count                8032  8032.000000
unique               8032          NaN
top     3.82485 100.89686          NaN
freq                    1          NaN
mean                  NaN     1.234437
std                   NaN     0.559892
min                   NaN     0.000000
25%                   NaN     1.000000
50%                   NaN     1.000000
75%                   NaN     1.000000
max                   NaN     4.000000
               point  frequency
0   3.53232 101.0906          4
1  2.97939 101.50694          4
2  3.14438 101.34838          4
3  3.16184 101.41519          4
4  3.13323 101.372

We can observe that most of the points (start/ end):
- mostly have a frequency of 1s where Q3 is up to 1
- quite a substantial amount of 2s at around 1000s though higher for the end points (which make sense since less unique end points)
- a small number of points with a frequency of 3 at around 400s to 500s
- lastly a very very minute number at a frequency of 4

=======================================================================================
# Point of Interest (POI) -- Petrol Stations
=======================================================================================

In [12]:
# get the petrol staitons
current_key = "Selangor_Petrol"
df_poi_petrol = dict_df_poi[current_key]
# then preprocess it
# note that it is possible for 2 petrol stations to be at the same point
df_poi_petrol = Lib_TrafficFlow.preprocess_dataframe_poi_petrol(df_poi_petrol)
dict_df_poi[current_key] = df_poi_petrol

In [13]:
# print out the details
current_key = "Selangor_Petrol"
df_poi_petrol = dict_df_poi[current_key]
Lib_TrafficFlow.print_details(df_poi_petrol)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 866 entries, 0 to 865
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   NO            866 non-null    int64   
 1   FEATURE CODE  866 non-null    category
 2   FEATURE NAME  866 non-null    category
 3   NAME          866 non-null    category
 4   TYPE          866 non-null    category
 5   STATE         866 non-null    category
 6   LAT           866 non-null    object  
 7   LONG          866 non-null    object  
 8   POINT         866 non-null    category
dtypes: category(6), int64(1), object(2)
memory usage: 127.2+ KB
None
                NO FEATURE CODE    FEATURE NAME   NAME      TYPE     STATE  \
count   866.000000          866             866    866       866       866   
unique         NaN            1               1    795         6         2   
top            NaN       BJ0330  Petrol Station  SHELL  PETRONAS  Selangor   
freq           NaN          86

We have a collection of petrol stations
- Total of 866 petrol stations
- 661 in Selangor, 205 in KL
- 6 different company, highest is Petronas with a frequency of 258
- it should be noted that there could be petrol stations that are at the same location (a total of 4 is found)


In [14]:
# get list of points that would be in the map
current_key = next(iter(dict_df_trafficflow.keys()))
print(current_key)
current_df = dict_df_trafficflow[current_key]
# we only want the end points and not start points
# also make sure that it is lat long
current_endpoints = Lib_TrafficFlow.get_vertices_from_dataframe(
    argv_df=current_df,
    argv_col="end_point",
    argv_before="",
    argv_after="",
    argv_type=None,
    argv_swap_latlong=False,
    argv_asPoint=True)
print(current_endpoints[0])



KL_Selangor_thursday_0800
POINT (3.08219 101.66824)


In [15]:
# add in the nearest points
current_key = "Selangor_Petrol"
df_poi_petrol = dict_df_poi[current_key]
current_points_nearest = []

current_petrol_points = df_poi_petrol["POINT"].tolist()
for current_petrol_point in current_petrol_points:
    current_point_nearest = Lib_TrafficFlow.get_nearest_vertex(argv_point=current_petrol_point, argv_vertices=current_endpoints, argv_point_type="string")
    current_point_nearest = str(current_point_nearest)
    current_point_nearest = Lib_TrafficFlow.extract_coordinate(argv_coordinate=current_point_nearest, argv_before="POINT (", argv_after=")")
    current_point_nearest = " ".join(current_point_nearest)
    current_points_nearest.append(current_point_nearest)

df_poi_petrol["POINT_nearest"] = current_points_nearest
dict_df_poi[current_key] = df_poi_petrol
Lib_TrafficFlow.print_details(df_poi_petrol)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 866 entries, 0 to 865
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   NO             866 non-null    int64   
 1   FEATURE CODE   866 non-null    category
 2   FEATURE NAME   866 non-null    category
 3   NAME           866 non-null    category
 4   TYPE           866 non-null    category
 5   STATE          866 non-null    category
 6   LAT            866 non-null    object  
 7   LONG           866 non-null    object  
 8   POINT          866 non-null    category
 9   POINT_nearest  866 non-null    object  
dtypes: category(6), int64(1), object(3)
memory usage: 134.0+ KB
None
                NO FEATURE CODE    FEATURE NAME   NAME      TYPE     STATE  \
count   866.000000          866             866    866       866       866   
unique         NaN            1               1    795         6         2   
top            NaN       BJ0330  Petrol Station  SH

=======================================================================================
# Shortest distance
=======================================================================================

In [16]:
# generate 1 graph
current_key = next(iter(dict_df_trafficflow.keys()))
print(current_key)
current_df = dict_df_trafficflow[current_key]
current_graph_length = Lib_TrafficFlow.generate_graph(
    argv_df=current_df,
    argv_source="start_point",
    argv_destination="end_point",
    argv_edge=["TMC_LE"]
)

KL_Selangor_thursday_0800


In [17]:
# generate random start points and end points
current_key = next(iter(dict_df_trafficflow.keys()))
current_df = dict_df_trafficflow[current_key]
current_selected_start = Lib_TrafficFlow.select_random(argv_values=current_df["start_point"].tolist(), argv_count=10)
current_selected_end = Lib_TrafficFlow.select_random(argv_values=current_df["end_point"].tolist(), argv_count=10)
print(current_selected_start)
print(current_selected_end)


['2.95842 101.75121', '3.14971 101.59946', '3.12845 101.7211', '3.10218 101.70868', '3.07108 101.50656', '3.13919 101.7204', '3.18927 101.72733', '3.13004 101.67039', '2.90081 101.48759', '3.39912 101.56049']
['3.10942 101.61715', '3.08095 101.67676', '3.10919 101.64471', '3.01503 101.43204', '2.8923 101.4779', '3.21865 101.51547', '3.14046 101.72028', '3.09359 101.7117', '2.91337 101.87255', '3.16825 101.73529']


In [18]:
# run dijkstra
current_distance, current_path = Lib_TrafficFlow.run_dijkstra(
    argv_graph=current_graph_length,
    argv_source="3.22209 101.43608",
    argv_target="2.98354 101.73192",
    argv_weight_edge="TMC_LE"
    )

print(current_distance)
print(len(current_path))
print(current_path)

80.75935
95
['3.22209 101.43608', '3.22236 101.43639', '3.22138 101.44654', '3.18904 101.43315', '3.17589 101.43442', '3.16177 101.43587', '3.14822 101.43771', '3.13618 101.43937', '3.12945 101.44025', '3.12322 101.44092', '3.1184 101.44124', '3.11275 101.44153', '3.10892 101.44191', '3.10084 101.44256', '3.09357 101.44359', '3.08449 101.44608', '3.07908 101.44761', '3.0744 101.44901', '3.06591 101.4514', '3.05316 101.45046', '3.05078 101.44984', '3.0503 101.45256', '3.05038 101.45269', '3.04866 101.45314', '3.04805 101.45217', '3.05341 101.46406', '3.05348 101.46751', '3.05327 101.46917', '3.05657 101.47596', '3.05902 101.48063', '3.06088 101.48195', '3.06318 101.48927', '3.06458 101.51473', '3.06737 101.53037', '3.07387 101.55096', '3.0745 101.55336', '3.08226 101.57739', '3.08507 101.58882', '3.09544 101.59077', '3.09324 101.59469', '3.08564 101.60531', '3.08407 101.60703', '3.0834 101.60674', '3.08421 101.61581', '3.08462 101.61942', '3.0849 101.62126', '3.08725 101.62609', '3.0988

In [19]:
# bellman ford
# no need because there isn't negative
# a greedy algorithm like dijsktra is alright

# floyd warshall
# not suitable as graph is sparse from our analytics here
# dijsktra is so much eaier to back track
# runtime too slow
"""
import networkx as networkx
bla = networkx.floyd_warshall_predecessor_and_distance(current_graph_length)
print(bla)
"""

# all pair dijsktra
# runtime too slow
# memory usage is very high at around 10gb while running
# and doesn't have have the path
"""
import networkx as networkx
bla = dict(networkx.all_pairs_dijkstra_path(current_graph_length))
print(bla)
"""

# single run dijsktra, only uses 300mb

'\nimport networkx as networkx\nbla = dict(networkx.all_pairs_dijkstra_path(current_graph_length))\nprint(bla)\n'

=======================================================================================
# Layered Graph
=======================================================================================

In [20]:
# get flow df
current_key = next(iter(dict_df_trafficflow.keys()))
print(current_key)
current_df = dict_df_trafficflow[current_key]
# get petrol df
current_key = "Selangor_Petrol"
df_poi_petrol = dict_df_poi[current_key]

# get list of intermediate node
current_points_intermediate = df_poi_petrol["POINT_nearest"]

# generate layered df
current_df_layered = Lib_TrafficFlow.generate_graph_layers(
    argv_df=current_df,
    argv_u="start_point",
    argv_v="end_point",
    argv_w="TMC_LE",
    argv_intermediate=current_points_intermediate,
    argv_intermediate_weight=0,
    argv_layers_count=2,
    argv_type="dataframe"
    )

# get details of df
Lib_TrafficFlow.print_details(current_df_layered)

KL_Selangor_thursday_0800
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20696 entries, 0 to 20695
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   start_point  20696 non-null  object 
 1   end_point    20696 non-null  object 
 2   TMC_LE       20696 non-null  float64
dtypes: float64(1), object(2)
memory usage: 485.2+ KB
None
                start_point            end_point        TMC_LE
count                 20696                20696  20696.000000
unique                16086                15488           NaN
top     3.12945 101.44025_0  3.12945 101.44025_1           NaN
freq                      7                    7           NaN
mean                    NaN                  NaN      0.812150
std                     NaN                  NaN      1.545708
min                     NaN                  NaN      0.000000
25%                     NaN                  NaN      0.061965
50%                     NaN   

In [21]:
# try to run dijkstra
# run dijkstra
current_graph_length_layered = Lib_TrafficFlow.generate_graph(
    argv_df=current_df_layered,
    argv_source="start_point",
    argv_destination="end_point",
    argv_edge=["TMC_LE"]
)
current_distance, current_path = Lib_TrafficFlow.run_dijkstra(
    argv_graph=current_graph_length_layered,
    argv_source="3.22209 101.43608_0",
    argv_target="2.98354 101.73192_1",
    argv_weight_edge="TMC_LE"
    )

print(current_distance)
print(len(current_path))
print(current_path)

80.75935
96
['3.22209 101.43608_0', '3.22236 101.43639_0', '3.22138 101.44654_0', '3.22138 101.44654_1', '3.18904 101.43315_1', '3.17589 101.43442_1', '3.16177 101.43587_1', '3.14822 101.43771_1', '3.13618 101.43937_1', '3.12945 101.44025_1', '3.12322 101.44092_1', '3.1184 101.44124_1', '3.11275 101.44153_1', '3.10892 101.44191_1', '3.10084 101.44256_1', '3.09357 101.44359_1', '3.08449 101.44608_1', '3.07908 101.44761_1', '3.0744 101.44901_1', '3.06591 101.4514_1', '3.05316 101.45046_1', '3.05078 101.44984_1', '3.0503 101.45256_1', '3.05038 101.45269_1', '3.04866 101.45314_1', '3.04805 101.45217_1', '3.05341 101.46406_1', '3.05348 101.46751_1', '3.05327 101.46917_1', '3.05657 101.47596_1', '3.05902 101.48063_1', '3.06088 101.48195_1', '3.06318 101.48927_1', '3.06458 101.51473_1', '3.06737 101.53037_1', '3.07387 101.55096_1', '3.0745 101.55336_1', '3.08226 101.57739_1', '3.08507 101.58882_1', '3.09544 101.59077_1', '3.09324 101.59469_1', '3.08564 101.60531_1', '3.08407 101.60703_1', '3.

In [22]:
# try to run dijsktra with 5 layers
current_df_layered = Lib_TrafficFlow.generate_graph_layers(
    argv_df=current_df,
    argv_u="start_point",
    argv_v="end_point",
    argv_w="TMC_LE",
    argv_intermediate=current_points_intermediate,
    argv_intermediate_weight=0,
    argv_layers_count=5,
    argv_type="dataframe"
    )
current_graph_length_layered = Lib_TrafficFlow.generate_graph(
    argv_df=current_df_layered,
    argv_source="start_point",
    argv_destination="end_point",
    argv_edge=["TMC_LE"]
)
current_distance, current_path = Lib_TrafficFlow.run_dijkstra(
    argv_graph=current_graph_length_layered,
    argv_source="3.22209 101.43608_0",
    argv_target="2.98354 101.73192_4",
    argv_weight_edge="TMC_LE"
    )

print(current_distance)
print(len(current_path))
print(current_path)

80.75935
99
['3.22209 101.43608_0', '3.22236 101.43639_0', '3.22138 101.44654_0', '3.22138 101.44654_1', '3.22138 101.44654_2', '3.22138 101.44654_3', '3.22138 101.44654_4', '3.18904 101.43315_4', '3.17589 101.43442_4', '3.16177 101.43587_4', '3.14822 101.43771_4', '3.13618 101.43937_4', '3.12945 101.44025_4', '3.12322 101.44092_4', '3.1184 101.44124_4', '3.11275 101.44153_4', '3.10892 101.44191_4', '3.10084 101.44256_4', '3.09357 101.44359_4', '3.08449 101.44608_4', '3.07908 101.44761_4', '3.0744 101.44901_4', '3.06591 101.4514_4', '3.05316 101.45046_4', '3.05078 101.44984_4', '3.0503 101.45256_4', '3.05038 101.45269_4', '3.04866 101.45314_4', '3.04805 101.45217_4', '3.05341 101.46406_4', '3.05348 101.46751_4', '3.05327 101.46917_4', '3.05657 101.47596_4', '3.05902 101.48063_4', '3.06088 101.48195_4', '3.06318 101.48927_4', '3.06458 101.51473_4', '3.06737 101.53037_4', '3.07387 101.55096_4', '3.0745 101.55336_4', '3.08226 101.57739_4', '3.08507 101.58882_4', '3.09544 101.59077_4', '3.

=======================================================================================
# EXPERIMENT
=======================================================================================

In [23]:
# experiment control
current_max_layers = 10
Lib_TrafficFlow.random_set_seed(1234)

# storing the results
current_results = []
current_result = []
current_result.append("current_key_traffic")
current_result.append("len_current_points_intermediate")
current_result.append("current_count_layer")
current_result.append("current_time_construct_graph_layers")
current_result.append("current_start_point")
current_result.append("current_end_point")
current_result.append("current_distance")
current_result.append("len_current_path")
current_result.append("current_time_dijkstra")
current_results.append(current_result)
filename_result = "C:\\Users\\Iam\\Dropbox\\_Research\\MIMOS_Location_2020\\_Codes_Ian_v20201102_2342\\result_MIMOS_20201123_1230.csv"
file_result = open(filename_result, "w+")
file_result.write(",".join(current_result))

# generate random start points and end points
current_key = next(iter(dict_df_trafficflow.keys()))
current_df = dict_df_trafficflow[current_key]
current_selected_start = Lib_TrafficFlow.select_random(argv_values=current_df["start_point"].tolist(), argv_count=10)
current_selected_end = Lib_TrafficFlow.select_random(argv_values=current_df["end_point"].tolist(), argv_count=10)

# get the intermediate vertices, using petrol
# but have variants to see how it grows
# in the future, we can explore different layer connectors, this require more understanding of start/ end by computing more first
current_key = "Selangor_Petrol"
df_poi_petrol = dict_df_poi[current_key]
current_points_intermediate = df_poi_petrol["POINT_nearest"]
current_points_intermediates = []
current_size = 1
while current_size < len(current_points_intermediate):
    current_points_intermediates.append(current_points_intermediate[:current_size])
    current_size = 2 * current_size
current_points_intermediates.append(current_points_intermediate)

# loop through the traffic flow
current_keys_traffic = dict_df_trafficflow.keys()
for current_key_traffic in current_keys_traffic:
    current_df_traffic = dict_df_trafficflow[current_key_traffic]
    # loop through the intermediate points
    for current_points_intermediate in current_points_intermediates:
        # loop through the layers
        for current_count_layer in range(2,current_max_layers+1,1):
            # generate the graph
            get_time()
            current_df_layered = Lib_TrafficFlow.generate_graph_layers(
                argv_df=current_df_traffic,
                argv_u="start_point",
                argv_v="end_point",
                argv_w="TMC_LE",
                argv_intermediate=current_points_intermediate,
                argv_intermediate_weight=0,
                argv_layers_count=current_count_layer,
                argv_type="dataframe"
            )
            current_time_construct_graph_layers = get_time()
            # loop through the points
            for current_start_point in current_selected_start:
                for current_end_point in current_selected_end:
                    # skip the one we have processed before
                    if current_start_point == current_end_point:
                        continue
                    # run dijkstra
                    # might have no path
                    get_time()
                    try:
                        current_distance, current_path = Lib_TrafficFlow.run_dijkstra(
                            argv_graph=current_graph_length_layered,
                            argv_source=str(current_start_point) + "_0",
                            argv_target=str(current_end_point) + "_" + str(current_count_layer-1),
                            argv_weight_edge="TMC_LE"
                            )
                    except:
                        current_distance = -1
                        current_path = []
                    current_time_dijkstra = get_time()
                    # add the results
                    current_result = []
                    current_result.append(str(current_key_traffic))
                    current_result.append(str(len(current_points_intermediate)))
                    current_result.append(str(current_count_layer))
                    current_result.append(str(current_time_construct_graph_layers))
                    current_result.append(str(current_start_point))
                    current_result.append(str(current_end_point))
                    current_result.append(str(current_distance))
                    current_result.append(str(len(current_path)))
                    current_result.append(str(current_time_dijkstra))
                    current_results.append(current_result)
                    file_result.write("\n")
                    file_result.write(",".join(current_result))

# close file
file_result.close()


=======================================================================================
# Results process
=======================================================================================

In [30]:
import pandas as pandas

filename_result = "result_MIMOS_20201123_1230.csv"
df_results = pandas.read_csv(filename_result)
Lib_TrafficFlow.print_details(df_results)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89100 entries, 0 to 89099
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   current_key_traffic                  89100 non-null  object 
 1   len_current_points_intermediate      89100 non-null  int64  
 2   current_count_layer                  89100 non-null  int64  
 3   current_time_construct_graph_layers  89100 non-null  float64
 4   current_start_point                  89100 non-null  object 
 5   current_end_point                    89100 non-null  object 
 6   current_distance                     89100 non-null  float64
 7   len_current_path                     89100 non-null  int64  
 8   current_time_dijkstra                89100 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 6.1+ MB
None
            current_key_traffic  len_current_points_intermediate  \
count                     89100

From the results
- we currently have 89100 results
    - from our combinations earlier

But not all routes can be found, so let us clear them first


In [32]:
# remove the ones with no path
df_results = df_results[df_results["current_distance"] != -1]
Lib_TrafficFlow.print_details(df_results)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22176 entries, 10 to 88599
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   current_key_traffic                  22176 non-null  object 
 1   len_current_points_intermediate      22176 non-null  int64  
 2   current_count_layer                  22176 non-null  int64  
 3   current_time_construct_graph_layers  22176 non-null  float64
 4   current_start_point                  22176 non-null  object 
 5   current_end_point                    22176 non-null  object 
 6   current_distance                     22176 non-null  float64
 7   len_current_path                     22176 non-null  int64  
 8   current_time_dijkstra                22176 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.7+ MB
None
            current_key_traffic  len_current_points_intermediate  \
count                     2217

This leave us with
- 22176 results left

And the statistics which we would use as well

=======================================================================================
# BREAK
=======================================================================================